In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd

import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '..', 'Model', 'Dev', 'MeanReversion')))
from MeanReversionAlg import MeanReversionTrader  # your module

In [ ]:

def loadPrices(fn):
    global nt, nInst
    df=pd.read_csv(fn, sep=r'\s+', header=None, index_col=None)
    (nt,nInst) = df.shape
    return (df.values).T

pricesFile="prices.txt"
prcAll = loadPrices(pricesFile)

prcTrain = prcAll[:, :450]
prcTestOnly= prcAll[:, 450:600]
prcTest= prcAll[:, :600]
prcValOnly= prcAll[:, 600:]
prcVal= prcAll[:, :]

In [ ]:

with open("market_condition/mean_reversion_params.pkl", "rb") as f:
    data = pickle.load(f)

beta_matrix = data["beta_matrix"]
alpha_matrix = data["alpha_matrix"]
pairs = data["pairs"]
if __name__ == "__main__":
    print(pairs)

In [ ]:
# --- Step 3: Run trader with ONE pair for now ---
num = 5
trader = MeanReversionTrader(pairs=[pairs[num]], beta_matrix=beta_matrix, alpha_matrix=alpha_matrix)

# Run simulation
trader.run(prcTrain)

# --- Step 4: Analyze Results ---
# Show trade log
log = trader.get_trade_log()
print(log)

# Plot the spread z-score evolution
# trader.plot_spread()
trader.plot_spread()

In [ ]:
completed_trades = log[log["Action"] == "Exit"].copy()

# Clean the Return column if it’s a string with "%"
completed_trades["Return"] = completed_trades["Return"].str.replace('%', '').astype(float)

# Compute mean return %
mean_return = completed_trades["Return"].mean()

print(f"Mean Return per Trade: {mean_return:.2f}%")


In [ ]:
completed_trades = log[log["Action"] == "Exit"].copy()

# Clean if stored as percentage strings (optional safeguard)
if completed_trades["Return"].dtype == object:
    completed_trades["Return"] = completed_trades["Return"].str.replace('%', '').astype(float)

total_trades = len(completed_trades)
wins = (completed_trades["Return"] > 0).sum()
win_rate = wins / total_trades if total_trades > 0 else np.nan

print(f"Total Trades: {total_trades}")
print(f"Wins: {wins}")
print(f"Win Rate: {win_rate:.2%}")

In [ ]:
def compute_trade_durations(log):
    """
    Computes duration stats of completed trades from the log.

    Returns:
        - durations (list of int)
        - mean duration
        - std deviation of duration
    """
    entries = log[log["Action"].str.startswith("Enter")].copy()
    exits = log[log["Action"] == "Exit"].copy()

    durations = []

    # Match entry and exit by asset pair
    for _, entry_row in entries.iterrows():
        day_entry = entry_row["Day"]
        a, b = entry_row["Asset A"], entry_row["Asset B"]

        # Find the first matching exit after this entry
        exit_match = exits[(exits["Asset A"] == a) & (exits["Asset B"] == b) & (exits["Day"] > day_entry)]
        if not exit_match.empty:
            day_exit = exit_match.iloc[0]["Day"]
            durations.append(day_exit - day_entry)

    if durations:
        durations = np.array(durations)
        mean_duration = durations.mean()
        std_duration = durations.std()
        print(f"Mean Trade Duration: {mean_duration:.2f} days")
        print(f"Std Dev of Trade Duration: {std_duration:.2f} days")
    else:
        mean_duration = std_duration = np.nan
        print("No completed trades to compute durations.")

    return durations, mean_duration, std_duration
durations, mean_duration, std_duration = compute_trade_durations(log)